In [ ]:
import os


In [2]:
! huggingface-cli download meta-llama/Llama-3.2-3B --max-workers 16

Fetching 15 files:   0%|                                 | 0/15 [00:00<?, ?it/s]Downloading 'generation_config.json' to '/root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B/blobs/2d73a6863086ff9d491c28e49df9fb697cd92c2b.incomplete'

generation_config.json: 100%|███████████████████| 185/185 [00:00<00:00, 865kB/s]
Download complete. Moving file to /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B/blobs/2d73a6863086ff9d491c28e49df9fb697cd92c2b

USE_POLICY.md: 0.00B [00:00, ?B/s]

USE_POLICY.md: 6.02kB [00:00, 1.01MB/s]               | 0.00/242 [00:00<?, ?B/s]
Download complete. Moving file to /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B/blobs/ac3c5f21b9779e3da0677d6d3c587778fe3a331e
.gitattributes: 1.52kB [00:00, 2.22MB/s]                                        
Download complete. Moving file to /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B/blobs/a6344aac8c09253b3b630fb776ae94478aa0275b
Fetching 15 files:   7%|█▋                      

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from rouge import Rouge
import sys
sys.path.append("/workspace/packges")
from MyDatasets import loading_dataset, collote_fn
import torch
import json

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args = {
        "aux_name": "meta-llama/Llama-3.1-8B-Instruct",
        "base_name": "meta-llama/Llama-2-7b-hf",
        "dataset": "/datasets/mandarjoshi/trivia_qa/rc.nocontext/rc_nocontext_validation.json",
        "batch_size": 1,
        "shuffle": True,
        "seed": 42,
        "loss_func": "all",
        "mode": "validation",
        "device": "cpu",
        "use_prompt": True
    }

In [3]:
aux_tokenizer = AutoTokenizer.from_pretrained(args["aux_name"])
aux = AutoModelForCausalLM.from_pretrained(args["aux_name"])
aux.eval()
aux_tokenizer.pad_token = aux_tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.17it/s]


In [4]:
dataset = loading_dataset(args["dataset"],use_prompt=args["use_prompt"])
dataloader = DataLoader(dataset, batch_size=args["batch_size"], shuffle=args["shuffle"], collate_fn=collote_fn)

In [5]:
dataset[0]

{'question': 'Answer the question directly within 5 words. Do NOT repeat the question or output any other words. Question:\nWho was the man behind The Chipmunks?',
 'answer': 'David Seville'}

In [7]:
rouge = Rouge()
for batched_data in tqdm(dataloader):
    inputs = aux_tokenizer(batched_data["question"], return_tensors="pt", padding=True, truncation=True)
    pred_ids = []
    # for i in range(3):
    #     outputs = aux(
    #         **inputs, 
    #         output_hidden_states=True, 
    #         output_attentions=True, 
    #         use_cache=True, 
    #         predicted_key_value=None, 
    #     )
        
    #     logits = outputs.logits[:, -1, :]
    #     pred_id = torch.argmax(logits, dim=-1)  # Greedy decoding
    #     if i == 0:
    #         pred_ids = pred_id.unsqueeze(1)
    #     else:
    #         pred_ids = torch.cat([pred_ids, pred_id.unsqueeze(1)], dim=-1)
        
    #     if all(pred_id == aux_tokenizer.eos_token_id):
    #         break
    #     # update the inputs
        
    #     inputs["input_ids"] = torch.cat([inputs["input_ids"], pred_id.unsqueeze(1)], dim=-1)
    #     inputs["attention_mask"] = torch.cat([inputs["attention_mask"], torch.ones_like(pred_id).unsqueeze(1)], dim=-1)
    #     print(inputs["input_ids"])
    #     print(inputs["attention_mask"])
    
    outputs = aux.generate(
        **inputs, 
        max_new_tokens=5,
        pad_token_id=aux_tokenizer.eos_token_id,
    )
    
    preds = aux_tokenizer.batch_decode(outputs[:,inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    refs = aux_tokenizer(batched_data["answer"], return_tensors="pt", padding=True, truncation=True)
    refs = aux_tokenizer.batch_decode(refs["input_ids"], skip_special_tokens=True)
    print(json.dumps(preds,indent=4))
    print(json.dumps(refs,indent=4))
    temp_scores = rouge.get_scores(hyps=preds, refs=refs, avg=True)
    temp_results = {key: value['f'] for key, value in temp_scores.items()}
    temp_results['avg'] = sum(temp_results.values()) / len(temp_results)
    print(f"current {args['mode']} Rouge-1: {temp_results['rouge-1']:>0.2f} Rouge-2: {temp_results['rouge-2']:>0.2f} Rouge-L: {temp_results['rouge-l']:>0.2f} average: {temp_results['avg']}\n")

  0%|          | 1/17944 [00:04<23:01:45,  4.62s/it]

[
    " 1768\nIn"
]
[
    "1768"
]
current validation Rouge-1: 0.67 Rouge-2: 0.00 Rouge-L: 0.67 average: 0.4444444414814815



  0%|          | 2/17944 [00:10<27:00:14,  5.42s/it]

[
    " Answer:\nLuxor."
]
[
    "Luxor"
]
current validation Rouge-1: 0.67 Rouge-2: 0.00 Rouge-L: 0.67 average: 0.4444444414814815



  0%|          | 3/17944 [00:14<24:33:13,  4.93s/it]

...

[
    " Genghis Khan."
]
[
    "Tartars"
]
current validation Rouge-1: 0.00 Rouge-2: 0.00 Rouge-L: 0.00 average: 0.0



  0%|          | 22/17944 [01:40<21:56:42,  4.41s/it]

[
    " Answer:\nThe Listener."
]
[
    "THE LISTENER"
]
current validation Rouge-1: 0.00 Rouge-2: 0.00 Rouge-L: 0.00 average: 0.0



  0%|          | 22/17944 [01:44<23:41:22,  4.76s/it]


KeyboardInterrupt: 